In [1]:
'''
    The initial point for this problem is originally set as [2,2,0]. However, the simplex code doesn't find the optimal point. 
    Thus, I choose another point [2,0,2] and checked that the code works to find the optimal point, which is consistent with CXVPY.
'''

import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt

# setup
max_iterations = 20

# Problem condition
xe = np.array([2,0,2,0,2,0,0], dtype = np.float32) # x = [x1,x2,x3,y1,y2,y3,y4]
c = np.array([1,1,-1,0,0,0,0], dtype = np.float32) # cTx = x1 + x2 - x3
A = np.array([
    [1,0,0,1,0,0,0],
    [0,1,0,0,1,0,0],
    [0,0,1,0,0,1,0],
    [1,1,1,0,0,0,1]
], dtype = np.float32)
b = np.array([2,2,2,4], dtype=np.float32)
B = np.array([0,1,2,3], dtype=np.int32)  # matching index order: [3,4,5]

In [2]:
from typing import List, Literal

class Simplex:
    def __init__(self, c:np.array, A:np.array, b:np.array, B:List):
        self.c = c
        self.A = A
        self.b = b
        self.B = B

    def choose_index(self, c:np.array, method:Literal["Bland", "Cost"]):

        if method == 'Bland':
            j = sorted(np.where(c < 0)[0])[0]
        else:
            j = np.where(c<0)[0][np.argmin(c[np.where(c < 0)[0]])]

        return j

    def solve(self, x_init:np.array, max_iterations: int, method:Literal["Bland", "Cost"]):

        n_iter = 0
        x = x_init.copy()
        status = None
        B = self.B
        c = self.c

        while n_iter < max_iterations:

            n_iter += 1

            A_B = self.A[:,B]
            c_B = c[B]

            p = la.solve(A_B.T, c_B)
            c_ = c - self.A.T @ p

            # optimal condition
            if np.all(c_ >= 0):
                status = "optimal"
                break

            j = self.choose_index(c_, method)

            # compute direction
            d = np.zeros((len(x),), dtype = np.float32)

            d[j] = 1
            d[B] = la.solve(A_B, -self.A[:, j].reshape(-1,))

            if np.all(d[B] >= 0):
                status = "Unbounded"
                break

            d_i = np.where(d[B] < 0)[0]

            i = B[d_i[np.argmin(-x[B[d_i]] / d[B[d_i]])]]
            step_size = np.min(-x[B[d_i]] / d[B[d_i]])
            status = "Not optimal"

            # Logging
            print("="*30)
            print("Iteration:{} | status:{}".format(n_iter, status))
            print("x:{}".format(x))
            print("cTx:{}".format(self.c @ x))
            print("A_B:{}".format(A_B))
            print("p:{}".format(p))
            print("reduced cost:{}".format(c_))
            print("Index j:{}".format(j))
            print("d_B:{}".format(d[B]))
            print("d:{}".format(d))
            print("Index i:{}".format(i))
            print("step size:{}".format(step_size))

            # update next x
            x += step_size * d

            # update new basis
            B[np.where(B == i)[0]] = j
            B = np.array(sorted(B))

        print("=" * 30)
        print("Process ended | status:{}".format(status))
        print("x:{}".format(x))
        print("cTx:{}".format(self.c @ x))
        print("A_B:{}".format(A_B))
        print("p:{}".format(p))
        print("reduced cost:{}".format(c_))
        print("Index j:{}".format(j))
        print("d_B:{}".format(d[B]))
        print("d:{}".format(d))
        print("Index i:{}".format(i))
        print("step size:{}".format(step_size))

        return x, B, status

In [3]:
# # Example problem
# x0 = np.array([0, 0, 0, 20, 20, 20], dtype = np.float32)
# B0 = np.array([3, 4, 5])

# c = np.array([-10, -12, -12, 0, 0, 0], dtype=np.float32)
# A = np.array([[1, 2, 2, 1, 0, 0], [2, 1, 2, 0, 1, 0], [2, 2, 1, 0, 0, 1]], dtype = np.float32)
# b = np.array([20, 20, 20], dtype=np.float32)

# solver = Simplex(c.copy(), A.copy(), b.copy(), B0.copy())
# x, _, status = solver.solve(x0.copy(), max_iterations, method = "Bland")

In [4]:
# Case 1: Bland's pivoting rule
solver = Simplex(c.copy(), A.copy(), b.copy(), B.copy())
x, _, status = solver.solve(xe.copy(), max_iterations, method = "Bland")

Iteration:1 | status:Not optimal
x:[2. 0. 2. 0. 2. 0. 0.]
cTx:0.0
A_B:[[1. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 1. 1. 0.]]
p:[ 0.  0. -2.  1.]
reduced cost:[ 0.  0.  0.  0.  0.  2. -1.]
Index j:6
d_B:[-1.  0.  0.  1.]
d:[-1.  0.  0.  1.  0.  0.  1.]
Index i:0
step size:2.0
Iteration:2 | status:Not optimal
x:[0. 0. 2. 2. 2. 0. 2.]
cTx:-2.0
A_B:[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 1. 0. 1.]]
p:[ 0.  1. -1.  0.]
reduced cost:[ 1.  0.  0.  0. -1.  1.  0.]
Index j:4
d_B:[-1.  0.  0.  1.]
d:[ 0. -1.  0.  0.  1.  0.  1.]
Index i:1
step size:0.0
Process ended | status:optimal
x:[0. 0. 2. 2. 2. 0. 2.]
cTx:-2.0
A_B:[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 1.]]
p:[ 0.  0. -1.  0.]
reduced cost:[1. 1. 0. 0. 0. 1. 0.]
Index j:4
d_B:[0. 0. 1. 1.]
d:[ 0. -1.  0.  0.  1.  0.  1.]
Index i:1
step size:0.0


In [5]:
print("x: {}".format(x))
print("cTx: {}".format(c @ x))
print("status: {}".format(status))

x: [0. 0. 2. 2. 2. 0. 2.]
cTx: -2.0
status: optimal


In [6]:
# Case 2: Most negative reduced cost index
solver = Simplex(c.copy(), A.copy(), b.copy(), B.copy())
x, _, status = solver.solve(xe.copy(), max_iterations, method="Cost")

Iteration:1 | status:Not optimal
x:[2. 0. 2. 0. 2. 0. 0.]
cTx:0.0
A_B:[[1. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 1. 1. 0.]]
p:[ 0.  0. -2.  1.]
reduced cost:[ 0.  0.  0.  0.  0.  2. -1.]
Index j:6
d_B:[-1.  0.  0.  1.]
d:[-1.  0.  0.  1.  0.  0.  1.]
Index i:0
step size:2.0
Iteration:2 | status:Not optimal
x:[0. 0. 2. 2. 2. 0. 2.]
cTx:-2.0
A_B:[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 1. 0. 1.]]
p:[ 0.  1. -1.  0.]
reduced cost:[ 1.  0.  0.  0. -1.  1.  0.]
Index j:4
d_B:[-1.  0.  0.  1.]
d:[ 0. -1.  0.  0.  1.  0.  1.]
Index i:1
step size:0.0
Process ended | status:optimal
x:[0. 0. 2. 2. 2. 0. 2.]
cTx:-2.0
A_B:[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 1.]]
p:[ 0.  0. -1.  0.]
reduced cost:[1. 1. 0. 0. 0. 1. 0.]
Index j:4
d_B:[0. 0. 1. 1.]
d:[ 0. -1.  0.  0.  1.  0.  1.]
Index i:1
step size:0.0


In [7]:
print("x: {}".format(x))
print("cTx: {}".format(c @ x))
print("status: {}".format(status))

x: [0. 0. 2. 2. 2. 0. 2.]
cTx: -2.0
status: optimal


In [10]:
# Compare the result with CXVPY
import cvxpy as cp

x = cp.Variable(7)
prob = cp.Problem(cp.Minimize(c.T @ x), [A @ x == b, -x <= 0])
prob.solve()

# Print result.
print("x:{}".format(np.round(x.value)))
print("cTx:{}".format(np.round(prob.value)))

x:[-0. -0.  2.  2.  2.  0.  2.]
cTx:-2.0
